In [9]:
import pandas as pd
import networkx as nx
import matplotlib as plt
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances, euclidean_distances

In [7]:
teams_df = pd.read_csv("cfb22.csv")
teams_df.head()

,Unnamed: 0,Off Rank,Team,Games,Win-Loss,Off Plays,Off Yards,Off Yards/Play,Off TDs,Off Yards per Game,...,Average Time of Possession per Game,Turnover Rank,Fumbles Recovered,Opponents Intercepted,Turnovers Gain,Fumbles Lost,Interceptions Thrown_y,Turnovers Lost,Turnover Margin,Avg Turnover Margin per Game
0,0,1,Tennessee (SEC),13,11-2,945,6832,7.23,78,525.5,...,25:02,10,11,11,22,8,3,11,11,0.85
1,1,2,Washington (Pac-12),13,11-2,975,6705,6.88,65,515.8,...,31:21,61,5,7,12,2,9,11,1,0.08
2,2,3,Southern California (Pac-12),14,11-3,988,7092,7.18,73,506.6,...,31:57,1,10,19,29,2,5,7,22,1.57
3,3,4,UCLA (Pac-12),13,9-4,934,6547,7.01,65,503.6,...,27:40,85,7,11,18,9,12,21,-3,-0.23
4,4,5,Georgia (SEC),15,15-0,1049,7517,7.17,75,501.1,...,33:37,66,7,12,19,11,7,18,1,0.07


In [16]:
def find_similar_teams(teams_df):
    # Subset the dataframe to only include the rushing and passing yard columns
    yards_df = teams_df[['Rush Yds', 'Pass Yards','Opp Pass Yds Allowed','Opp Rush Yards Alloweed']]
    
    # Compute the cosine similarity matrix between all teams
    similarity_matrix = cosine_similarity(yards_df)
    
    # Convert the similarity matrix to a dataframe
    similarity_df = pd.DataFrame(similarity_matrix, columns=teams_df.index, index=teams_df.index)
    
    # Sort the similarity dataframe by the values in the first column (which corresponds to each team's similarity to themselves)
    similarity_df = similarity_df.sort_values(by=teams_df.index[0], ascending=False)
    
    # Return the top 5 most similar teams
    return similarity_df.index[1:6]

In [17]:
similar_teams = find_similar_teams(teams_df)
print(similar_teams)

Int64Index([37, 22, 38, 47, 33], dtype='int64')


In [18]:
G = nx.Graph()

In [21]:
for team in similar_teams:
    G.add_node(team)

for i in range(len(similar_teams)):
    for j in range(i+1, len(similar_teams)):
        team1 = similar_teams[i]
        team2 = similar_teams[j]
        distance = 1 - cosine_similarity(teams_df.loc[team1, ['Rush Yds', 'Pass Yards','Opp Pass Yds Allowed','Opp Rush Yards Alloweed']].values.reshape(1, -1),
                                         teams_df.loc[team2, ['Rush Yds', 'Pass Yards','Opp Pass Yds Allowed','Opp Rush Yards Alloweed']].values.reshape(1, -1))
        G.add_edge(team1, team2, weight=distance)
        
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True)
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
plt.show()

ValueError: shape mismatch: value array of shape (10,1,1) could not be broadcast to indexing result of shape (10,)